In [1]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
pip install accelerate -U

In [3]:
# Install the necessary libraries
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('Kaludi/Customer-Support-Responses')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/74 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [7]:
dataset["train"]["query"][:10]

["My order hasn't arrived yet.",
 'I received a damaged product.',
 'I need to return an item.',
 'I want to change my shipping address.',
 'I have a question about my bill.',
 'How do I cancel my subscription?',
 'Can I get a refund for my purchase?',
 "I'd like to track my order.",
 'My account has been locked.',
 "I can't find the item I'm looking for."]

In [8]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Add a special pad token
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

def preprocess_data(examples):
    inputs = examples['query']
    outputs = examples['response']
    concatenated_inputs=[]
    for i in range(len(inputs)):
        concatenated_input="Query:"+inputs[i]
        concatenated_input+=" Response:"+outputs[i]
        concatenated_inputs.append(concatenated_input)
    encoded_input=tokenizer(concatenated_inputs,truncation=True,max_length=512,padding='max_length')

    return {
        'input_ids': encoded_input["input_ids"],
        'attention_mask': encoded_input['attention_mask'],
        'labels': encoded_input['input_ids']
    }

# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [9]:
#this is an example of an encoded input
print(tokenized_datasets["train"]["input_ids"][0])

[20746, 25, 3666, 1502, 5818, 470, 5284, 1865, 13, 18261, 25, 1135, 16521, 329, 262, 31818, 13, 1680, 345, 3387, 2148, 534, 1502, 1271, 523, 356, 460, 9161, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,

In [10]:
#to decode a single input sequence use decode
tokenizer.decode(tokenized_datasets["train"]["input_ids"][0],skip_special_tokens=True)

"Query:My order hasn't arrived yet. Response:We apologize for the inconvenience. Can you please provide your order number so we can investigate?"

In [11]:
#to decode more inputs use batch_decode like this
tokenizer.batch_decode(tokenized_datasets["train"]["input_ids"][:5],skip_special_tokens=True)

["Query:My order hasn't arrived yet. Response:We apologize for the inconvenience. Can you please provide your order number so we can investigate?",
 'Query:I received a damaged product. Response:We apologize for the inconvenience. Can you please provide a photo of the damaged product so we can assist you further?',
 'Query:I need to return an item. Response:Certainly. Please provide your order number and reason for return, and we will provide you with instructions on how to proceed.',
 "Query:I want to change my shipping address. Response:No problem. Can you please provide your order number and the new shipping address you'd like to use?",
 "Query:I have a question about my bill. Response:We'd be happy to help. Can you please provide your account number and a brief description of your question?"]

In [12]:
type(tokenized_datasets)

datasets.dataset_dict.DatasetDict

In [13]:
from sklearn.model_selection import train_test_split

# Convert datasets to lists for train_test_split
train_dataset_list = tokenized_datasets['train'].shuffle(seed=42).select(range(len(tokenized_datasets['train'])))

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset_list))
val_size = len(train_dataset_list) - train_size

train_dataset = train_dataset_list.select(range(train_size))
val_dataset = train_dataset_list.select(range(train_size, len(train_dataset_list)))

In [14]:
train_dataset_list

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 74
})

In [15]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

# Load the model
model = AutoModelForCausalLM.from_pretrained('gpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    num_train_epochs=10.0,
    weight_decay=0.01,
    logging_steps=10,
    logging_strategy="steps",
    report_to="none"
)

# Initialize the Trainer with the validation dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.253158
2,2.650500,0.204200
3,0.237000,0.160851
4,0.172400,0.131706
5,0.134200,0.114728
6,0.134200,0.106576
7,0.105700,0.102387
8,0.097300,0.099806
9,0.095000,0.098554
10,0.089500,0.098166


TrainOutput(global_step=80, training_loss=0.447706912457943, metrics={'train_runtime': 98.0683, 'train_samples_per_second': 6.016, 'train_steps_per_second': 0.816, 'total_flos': 154162298880000.0, 'train_loss': 0.447706912457943, 'epoch': 10.0})

In [17]:
validation=tokenizer.batch_decode(val_dataset["input_ids"],skip_special_tokens=True)
def extract_queries(val_text):
    begin_response=val_text.find("Response:")-1 # because there is a space before the word Response
    return val_text[:begin_response+len(" Response:")]
def extract_responses(val_text):
    begin_response=val_text.find("Response:")-1 # because there is a space before the word Response
    return val_text[begin_response+len(" Response:"):]
real_queries=list(map(extract_queries,validation))
real_responses=list(map(extract_responses,validation))

In [18]:
real_queries

['Query:What are your customer service hours? Response:',
 'Query:Can I pay with a gift card online? Response:',
 'Query:I was charged twice for my order. Response:',
 'Query:How do I apply for a job at your company? Response:',
 "Query:My coupon isn't working. Response:",
 'Query:Do you offer free shipping? Response:',
 "Query:I can't find my order confirmation email. Response:",
 "Query:I haven't received a response to my email inquiry. Response:",
 'Query:How do I use a gift card in-store? Response:',
 'Query:What is the status of my warranty claim? Response:',
 'Query:Can I change the size of an item in my order? Response:',
 'Query:How do I report a problem with your website? Response:',
 'Query:Can you help me with a product recommendation? Response:',
 'Query:Can I place a custom order? Response:',
 'Query:My account has been locked. Response:']

In [19]:
real_responses

['Our customer service team is available 24/7 to assist you. Please feel free to reach out with any questions or concerns.',
 "Yes, you can use gift cards for online purchases. Can you please provide the gift card number and the product name or SKU you're interested in?",
 'We apologize for the inconvenience. Can you please provide your order number so we can investigate and resolve the issue?',
 'Thank you for your interest. Can you please provide your email address so we can send you information on how to apply?',
 "We apologize for the inconvenience. Can you please provide the coupon code and the product name or SKU you're trying to apply it to?",
 'We do offer free shipping on select orders. Can you please provide the product name or SKU and your shipping destination so we can check if your order qualifies?',
 'We apologize for the inconvenience. Can you please provide your account email or order number so we can resend the confirmation email?',
 'We apologize for the delay. Can yo

In [20]:
import torch
import evaluate
bleu=evaluate.load("bleu")
encode_validation_queries=tokenizer(real_queries,max_length=512,truncation=True,padding='max_length')
#encode_validation_queries
input_ids=torch.tensor(encode_validation_queries["input_ids"],device='cuda')
attention_mask=torch.tensor(encode_validation_queries["attention_mask"],device='cuda')
full_decoded=tokenizer.batch_decode(model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=1024),skip_special_tokens=True)
predictions=[]
for i,query_response in enumerate(full_decoded):
    #print(query_response)
    find_begin_response=query_response.find("Response:")
    response=query_response[find_begin_response+len("Response:"):].strip()
    predictions.append(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [21]:
predictions

["We'd be happy to help. Can you please provide your email address so we can send you an email when we can provide you with the exact hours you'd like to schedule your appointment?",
 "We'd be happy to help. Can you please provide your gift card number or email address so we can provide you with the gift card you're looking for?",
 "We apologize for the inconvenience. Can you please provide the product name, SKU, or SKU you're interested in?",
 "We'd be happy to help. Can you please provide your email address so we can send you an email when we can provide you with the job title or position title?",
 'We apologize for the inconvenience. Can you please provide the product name or SKU so we can check the availability of the product?',
 'We do offer free shipping on orders over $50. Can you please provide your order number so we can provide you with the shipping information?',
 'We apologize for the inconvenience. Can you please provide your order confirmation email so we can check your a

In [22]:
real_responses

['Our customer service team is available 24/7 to assist you. Please feel free to reach out with any questions or concerns.',
 "Yes, you can use gift cards for online purchases. Can you please provide the gift card number and the product name or SKU you're interested in?",
 'We apologize for the inconvenience. Can you please provide your order number so we can investigate and resolve the issue?',
 'Thank you for your interest. Can you please provide your email address so we can send you information on how to apply?',
 "We apologize for the inconvenience. Can you please provide the coupon code and the product name or SKU you're trying to apply it to?",
 'We do offer free shipping on select orders. Can you please provide the product name or SKU and your shipping destination so we can check if your order qualifies?',
 'We apologize for the inconvenience. Can you please provide your account email or order number so we can resend the confirmation email?',
 'We apologize for the delay. Can yo

In [23]:
bleu.compute(predictions=predictions,references=real_responses)

{'bleu': 0.26770663342674006,
 'precisions': [0.5014492753623189,
  0.3253012048192771,
  0.2507836990595611,
  0.19281045751633988],
 'brevity_penalty': 0.8983043243685622,
 'length_ratio': 0.9031413612565445,
 'translation_length': 345,
 'reference_length': 382}

In [24]:
demo_query="Why does it take so long?"
input_query="Query: "+demo_query+" Response:"
encoded_input=tokenizer(input_query)
input_ids=torch.tensor([encoded_input["input_ids"]],device='cuda')
attention_mask=torch.tensor(encoded_input["attention_mask"],device='cuda')
response=model.generate(input_ids=input_ids,max_length=512)
query_and_response=tokenizer.decode(response[0],skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
find_response=query_and_response.find("Response:")
query_and_response[find_response+len("Response:"):]

"We'd be happy to help. Can you please provide your email address so we can send you an estimate of how long it takes to process your order?"

In [26]:
while True:
    demo_query=input("Customer: ")
    if(demo_query.lower()=="quit"):
        break
    gpt2_input_query="Query: "+demo_query+" Response:"
    encoded_input=tokenizer(gpt2_input_query)
    input_ids=torch.tensor([encoded_input["input_ids"]],device='cuda')
    attention_mask=torch.tensor(encoded_input["attention_mask"],device='cuda')
    response=model.generate(input_ids=input_ids,max_length=512)
    query_and_response=tokenizer.decode(response[0],skip_special_tokens=True)
    find_response=query_and_response.find("Response:")
    response=query_and_response[find_response+len("Response:"):]
    print("Response: {}".format(response))

Customer: Where is my product?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response:  We're sorry to hear about your product. Can you please provide your product name, SKU, or SKU so we can assist you?
Customer: quit
